# Форматы данных (1)

Материалы:
* Макрушин С.В. "Лекция 4: Форматы данных"
* https://docs.python.org/3/library/json.html
* https://docs.python.org/3/library/pickle.html
* https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

1. Вывести все адреса электронной почты, содержащиеся в адресной книге `addres-book.json`

2. Вывести телефоны, содержащиеся в адресной книге `addres-book.json`

3. По данным из файла `addres-book-q.xml` сформировать список словарей с телефонами каждого из людей. 

## Лабораторная работа №4

### JSON

1.1 Считайте файл `contributors_sample.json`. Воспользовавшись модулем `json`, преобразуйте содержимое файла в соответствующие объекты python. Выведите на экран информацию о первых 3 пользователях.

In [2]:
import json

with open('data/contributors_sample.json') as f:
    contributors = json.load(f)

for i in range(3):
    print(contributors[i])


{'username': 'uhebert', 'name': 'Lindsey Nguyen', 'sex': 'F', 'address': '01261 Cameron Spring\nTaylorfurt, AK 97791', 'mail': 'jsalazar@gmail.com', 'jobs': ['Energy engineer', 'Engineer, site', 'Environmental health practitioner', 'Biomedical scientist', 'Jewellery designer'], 'id': 35193}
{'username': 'vickitaylor', 'name': 'Cheryl Lewis', 'sex': 'F', 'address': '66992 Welch Brooks\nMarshallshire, ID 56004', 'mail': 'bhudson@gmail.com', 'jobs': ['Music therapist', 'Volunteer coordinator', 'Designer, interior/spatial'], 'id': 91970}
{'username': 'sheilaadams', 'name': 'Julia Allen', 'sex': 'F', 'address': 'Unit 1632 Box 2971\nDPO AE 23297', 'mail': 'darren44@yahoo.com', 'jobs': ['Management consultant', 'Engineer, structural', 'Lecturer, higher education', 'Theatre manager', 'Designer, textile'], 'id': 1848091}


1.2 Выведите уникальные почтовые домены, содержащиеся в почтовых адресах людей

In [4]:
emails = set()
for contributor in contributors:
    emails.add(contributor['mail'].split('@')[1])
print(emails)

{'yahoo.com', 'hotmail.com', 'gmail.com'}


1.3 Напишите функцию, которая по `username` ищет человека и выводит информацию о нем. Если пользователь с заданным `username` отсутствует, возбудите исключение `ValueError`

In [5]:
def find_contributor_by_username(username):
    for contributor in contributors:
        if contributor['username'] == username:
            return contributor
    raise ValueError('User not found')

username = 'jane_doe'
try:
    contributor = find_contributor_by_username(username)
    print(contributor)
except ValueError as e:
    print(e)

User not found


1.4 Посчитайте, сколько мужчин и женщин присутсвует в этом наборе данных.

In [7]:
male_count = 0
female_count = 0
for contributor in contributors:
    if contributor['sex'] == 'M':
        male_count += 1
    elif contributor['sex'] == 'F':
        female_count += 1
print(f"Male count: {male_count}")
print(f"Female count: {female_count}")

Male count: 2064
Female count: 2136


1.5 Создайте `pd.DataFrame` `contributors`, имеющий столбцы `id`, `username` и `sex`.

In [8]:
import pandas as pd

contributors_dict = {'id': [], 'username': [], 'sex': []}
for contributor in contributors:
    contributors_dict['id'].append(contributor['id'])
    contributors_dict['username'].append(contributor['username'])
    contributors_dict['sex'].append(contributor['sex'])
contributors_df = pd.DataFrame(contributors_dict)
print(contributors_df)


           id       username sex
0       35193        uhebert   F
1       91970    vickitaylor   F
2     1848091    sheilaadams   F
3       50969       nicole82   F
4      676820         jean67   M
...       ...            ...  ..
4195   423555  stevenspencer   F
4196    35251      rwilliams   M
4197   135887      lmartinez   F
4198   212714     brendahill   M
4199   344321       mistyray   F

[4200 rows x 3 columns]


1.6 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Объедините `recipes` с таблицей `contributors` с сохранением строк в том случае, если информация о человеке отсутствует в JSON-файле. Для скольких человек информация отсутствует? 

In [9]:
recipes = pd.read_csv('data/recipes_sample.csv')
merged_df = pd.merge(recipes, contributors_df, on='id', how='left')
print(f"Number of missing contributors: {merged_df['username'].isnull().sum()}")

Number of missing contributors: 29830


### pickle

2.1 На основе файла `contributors_sample.json` создайте словарь следующего вида: 
```
{
    должность: [список username людей, занимавших эту должность]
}
```

In [17]:
import json

with open('data/contributors_sample.json', 'r') as f:
    contributors = json.load(f)

job_people = {}

for contributor in contributors:
    for job in contributor['jobs']:
        if job not in job_people:
            job_people[job] = [contributor['username']]
        else:
            job_people[job].append(contributor['username'])


2.2 Сохраните результаты в файл `job_people.pickle` и в файл `job_people.json` с использованием форматов pickle и JSON соответственно. Сравните объемы получившихся файлов. При сохранении в JSON укажите аргумент `indent`.

In [18]:
import json
import pickle

# Сохранение в файл job_people.pickle
with open('job_people.pickle', 'wb') as f:
    pickle.dump(job_people, f)

# Сохранение в файл job_people.json с использованием формата JSON
with open('job_people.json', 'w') as f:
    json.dump(job_people, f, indent=4)


2.3 Считайте файл `job_people.pickle` и продемонстрируйте, что данные считались корректно. 

In [19]:
import pickle

# Чтение данных из файла job_people.pickle
with open('job_people.pickle', 'rb') as f:
    job_people_pickle = pickle.load(f)

# Проверка, что данные считались корректно
print(job_people_pickle == job_people)


True


### XML

3.1 По данным файла `steps_sample.xml` сформируйте словарь с шагами по каждому рецепту вида `{id_рецепта: ["шаг1", "шаг2"]}`. Сохраните этот словарь в файл `steps_sample.json`

In [30]:
import xml.etree.ElementTree as ET
import json

# Парсинг XML-файла
tree = ET.parse('data/steps_sample.xml')
root = tree.getroot()

steps_dict = {}

# Проход по всем элементам рецептов и сохранение шагов в словарь
for recipe in root.iter('recipe'):
    recipe_id = recipe.get('id')
    steps = [step.text for step in recipe.findall('./recipemethod/step')]
    steps_dict[recipe_id] = steps

# Сохранение словаря в файл JSON
with open('steps_sample.json', 'w') as f:
    json.dump(steps_dict, f)


3.2 По данным файла `steps_sample.xml` сформируйте словарь следующего вида: `кол-во_шагов_в_рецепте: [список_id_рецептов]`

In [31]:
steps_count_dict = {}

# Проход по всем элементам рецептов и сохранение количества шагов в словарь
for recipe in root.iter('recipe'):
    recipe_id = recipe.get('id')
    steps_count = len(recipe.findall('./recipemethod/step'))
    if steps_count in steps_count_dict:
        steps_count_dict[steps_count].append(recipe_id)
    else:
        steps_count_dict[steps_count] = [recipe_id]

# Сохранение словаря в файл JSON
with open('steps_count_sample.json', 'w') as f:
    json.dump(steps_count_dict, f)


3.3 Получите список рецептов, в этапах выполнения которых есть информация о времени (часы или минуты). Для отбора подходящих рецептов обратите внимание на атрибуты соответствующих тэгов.

In [32]:
recipes_with_time = []

for recipe in root.iter('recipe'):
    for step in recipe.iter('step'):
        time = step.get('time')
        if time and ('h' in time or 'min' in time):
            recipes_with_time.append(recipe.get('id'))
            break


3.4 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Для строк, которые содержат пропуски в столбце `n_steps`, заполните этот столбец на основе файла  `steps_sample.xml`. Строки, в которых столбец `n_steps` заполнен, оставьте без изменений.

In [33]:
import pandas as pd

# Загрузка данных из файла CSV
recipes_df = pd.read_csv('data/recipes_sample.csv')

# Функция для получения количества шагов в рецепте по его ID
def get_steps_count(recipe_id):
    try:
        return len(steps_dict[str(recipe_id)])
    except KeyError:
        return pd.NA

# Заполнение пропущенных значений в столбце n_steps
steps_dict = json.load(open('steps_sample.json', 'r'))

recipes_df['n_steps'] = recipes_df.apply(lambda row: get_steps_count(row['id']) if pd.isna(row['n_steps']) else row['n_steps'], axis=1)


3.5 Проверьте, содержит ли столбец `n_steps` пропуски. Если нет, то преобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_filled_nsteps.csv`

In [35]:
# Проверка наличия пропущенных значений в столбце n_steps
if recipes_df['n_steps'].isna().sum() == 0:
    # Преобразование столбца n_steps к целочисленному типу
    recipes_df['n_steps'] = recipes_df['n_steps'].astype(int)
    # Сохранение результатов в файл CSV
    recipes_df.to_csv('recipes_sample_with_filled_nsteps.csv', index=False)
else:
    print('Столбец n_steps содержит пропущенные значения')


Столбец n_steps содержит пропущенные значения
